### ショートカット（その1）

In [ ]:
import pandas as pd
import numpy as np

pred1_path = '/content/drive/MyDrive/GradReserch/hotpot-master/kaiseki/data/dev_distractor_pred_RRRRRR.json'
pred2_path = '/content/drive/MyDrive/GradReserch/hotpot-master/kaiseki/data/dev_distractor_pred_RRLLLL.json'
gold_path = '/content/drive/MyDrive/GradReserch/hotpot-master/kaiseki/data/hotpot_dev_distractor_v1.json'

pred1 = pd.read_json(pred1_path)
pred2 = pd.read_json(pred2_path)
gold  = pd.read_json(gold_path)

In [ ]:
print(pred1)

                                     answer  \
5ae66d515542992ae0d16346               1927   
5a83aaeb5542996488c2e483         Graduation   
5a8c66db554299585d9e3683               2017   
5ae16ea85542990adbacf790               1975   
5ac10832554299012d1db687                 22   
...                                     ...   
5abdb9775542993f32c2a01b  Prudential Center   
5a8fb3af5542997ba9cb32ee                yes   
5adf99065542992d7e9f93b0                45%   
5ac3d667554299076e296ca5        Appenzeller   
5ae3fe635542996836b02c07    Martin O'Malley   

                                                                         sp  
5ae66d515542992ae0d16346  [[I'll Keep the Lovelight Burning, 0], [Patti ...  
5a83aaeb5542996488c2e483  [[Graduation (album), 0], [The College Dropout...  
5a8c66db554299585d9e3683  [[Eleventh Mom, 0], [Mom (film), 0], [Mom (fil...  
5ae16ea85542990adbacf790  [[UVF Mid-Ulster Brigade, 5], [Miami Showband ...  
5ac10832554299012d1db687  [[John Frink, 1], [

In [ ]:
print(gold)

                           _id                            answer  \
0     5a8b57f25542995d1e6f1371                               yes   
1     5a8c7595554299585d9e36b6                 Chief of Protocol   
2     5a85ea095542994775f606a8                         Animorphs   
3     5adbf0a255429947ff17385a                                no   
4     5a8e3ea95542995a26add48d  Greenwich Village, New York City   
...                        ...                               ...   
7400  5ab92307554299753720f72d                     Yasir Hussain   
7401  5abba3b1554299642a094aed                                no   
7402  5a8173fa554299260e20a28e                               yes   
7403  5a8caf1d554299585d9e3720                               yes   
7404  5ac132a755429964131be17c            Norwood, Massachusetts   

                                               question  \
0     Were Scott Derrickson and Ed Wood of the same ...   
1     What government position was held by the woman...   
2     

In [ ]:
print(pred1_path[-11:-5])

RRRRRR


In [ ]:
print('pred1: {}'.format(len(pred1)))
print('pred2: {}'.format(len(pred2)))
print('gold : {}'.format(len(gold)))

pred1: 7376
pred2: 7376
gold : 7405


In [ ]:
df = pd.merge(pred1, pred2, left_index=True, right_index=True)

In [ ]:
df_ = pd.merge(df, gold.set_index('_id').drop(columns='level'), left_index=True, right_index=True)

In [ ]:
deff_ans = df_
deff_sp  = df_
len(deff_ans)
len(deff_sp)
print('deff_ans: {}\ndeff_sp : {}'.format(len(deff_ans), len(deff_sp)))

deff_ans: 7376
deff_sp : 7376


In [ ]:
choices = [1]
conditions = [
    deff_ans['answer_x'] == deff_ans['answer']
]
deff_ans['old_em'] = np.select(conditions, choices, default=0)

In [ ]:
choices = [1]
conditions = [
    deff_ans['answer_y'] == deff_ans['answer']
]
deff_ans['new_em'] = np.select(conditions, choices, default=0)

In [ ]:
choices = [1]
conditions = [
    deff_sp['sp_x'] == deff_sp['supporting_facts']
]
deff_sp['old_sp_em'] = np.select(conditions, choices, default=0)

In [ ]:
choices = [1]
conditions = [
    deff_sp['sp_y'] == deff_sp['supporting_facts']
]
deff_sp['new_sp_em'] = np.select(conditions, choices, default=0)

In [ ]:
better_answer = deff_ans[deff_ans['old_em']==0].drop(columns='old_em')
better_answer = better_answer[better_answer['new_em']==1].drop(columns='new_em')
print('better_answer: {} ({:.2f}%)'.format(len(better_answer), float(100*len(better_answer)/len(df))))
better_answer.head()

# RRRRRR -> RRLLLL にすることでanswer正解した問題

better_answer: 921 (12.49%)


,answer_x,sp_x,answer_y,sp_y,answer,question,supporting_facts,context,type,old_sp_em,new_sp_em
5a85b0c85542994c784ddb40,University,"[[2017 Texas Tech Red Raiders baseball team, 0...",Texas Tech University,"[[2017 Texas Tech Red Raiders baseball team, 0...",Texas Tech University,What institution does 2017 Texas Tech Red Raid...,"[[2017 Texas Tech Red Raiders baseball team, 0...","[[2017 Texas Tech Red Raiders baseball team, [...",bridge,0,0
5a8790785542993e715abfac,Erwin McNair,"[[Ronald McNair, 0], [Ronald McNair, 2]]",Cheryl,"[[Ronald McNair, 0], [Ronald McNair, 2]]",Cheryl,What was the first name of the wife of the nam...,"[[Dr. Ronald E. McNair Academic High School, 0...","[[Infinity Institute, [Infinity Institute is a...",bridge,0,0
5adfc97255429906c02daa41,second tier,"[[Notts County F.C., 3], [Bolton Wanderers F.C...",second,"[[Bolton Wanderers F.C., 1]]",second,In which tier of the English football league s...,"[[1894 FA Cup Final, 0], [Bolton Wanderers F.C...","[[1894 FA Cup Final, [The 1894 FA Cup Final wa...",bridge,0,0
5abbb42555429931dba144af,Jean-Paul Sartre,"[[Jean-Paul Sartre Experience, 0], [Jean-Paul ...",George Bernard Shaw,"[[Jean-Paul Sartre, 0], [George Bernard Shaw, 0]]",George Bernard Shaw,Jean-Paul Sartre or George Bernard Shaw have ...,"[[Jean-Paul Sartre, 0], [George Bernard Shaw, 0]]","[[Saint Genet, [Saint Genet, Actor and Martyr ...",comparison,0,1
5abe9c1a5542993f32c2a183,Tillandsia,"[[Los Andes Peruvian University, 3], [Alerce, ...",Robert FitzRoy,"[[Los Andes Peruvian University, 3], [Alerce, 0]]",Robert FitzRoy,Who was a species of Alerce tree native to the...,"[[Alerce, 0], [Fitzroya, 1], [Fitzroya, 2]]","[[Zebrawood, [The name zebrawood is used to de...",bridge,0,0


In [ ]:
worse_answer = deff_ans[deff_ans['old_em']==1].drop(columns='old_em')
worse_answer = worse_answer[worse_answer['new_em']==0].drop(columns='new_em')
print('worse_answer: {} ({:.2f}%)'.format(len(worse_answer), float(100*len(worse_answer)/len(df))))
worse_answer.head()

# RRRRRR -> RRLLLL にすることでanswer苦手になった問題

worse_answer: 558 (7.57%)


,answer_x,sp_x,answer_y,sp_y,answer,question,supporting_facts,context,type,old_sp_em,new_sp_em
5a8e43455542990e94052aa1,no,"[[Greg Costikyan, 0]]",yes,"[[Greg Costikyan, 0], [John Dolmayan, 0], [Joh...",no,Did Greg Costikyan have the same profession as...,"[[John Dolmayan, 0], [Greg Costikyan, 0]]","[[MadMaze, [MadMaze is an online video game de...",comparison,0,0
5a7f40de5542992e7d278cc8,"Richard ""Rick"" Ducommun","[[The 'Burbs, 0], [Rick Ducommun, 0]]",Carrie Fisher,"[[The 'Burbs, 0], [Rick Ducommun, 0], [Carrie ...","Richard ""Rick"" Ducommun",What star if the Burbs was Canadian?,"[[The 'Burbs, 0], [Rick Ducommun, 0]]","[[General Campaign Star, [The General Campaign...",bridge,1,0
5adbf84555429947ff17387c,no,"[[James Norman Hall, 0], [Adolfo Bioy Casares,...",yes,"[[James Norman Hall, 0], [Adolfo Bioy Casares,...",no,Are both Adolfo Bioy Casares and James Norman ...,"[[Adolfo Bioy Casares, 0], [James Norman Hall,...","[[La aventura de un fotógrafo en La Plata, [La...",comparison,0,0
5adc8c545542994734353734,yes,"[[In the Bedroom, 1], [Wilco Melissant, 0], [T...",no,"[[Wilco Melissant, 0], [Todd Field, 0]]",yes,Are Wilco Melissant and Todd Field both involv...,"[[Wilco Melissant, 0], [Todd Field, 0]]","[[Little Children (film), [Little Children is ...",comparison,0,1
5a889d18554299206df2b310,band,"[[Pinhead Gunpowder, 0], [Billie Joe Armstrong...",rock band,"[[Pinhead Gunpowder, 0], [Billie Joe Armstrong...",band,What kind of group does Pinhead Gunpowder and ...,"[[Pinhead Gunpowder, 0], [Billie Joe Armstrong...","[[Pinhead Gunpowder, [Pinhead Gunpowder is an ...",bridge,1,1


In [ ]:
normal_answer = deff_ans[deff_ans['old_em']==0].drop(columns='old_em')
normal_answer = normal_answer[normal_answer['new_em']==0].drop(columns='new_em')
print('normal_answer: {} ({:.2f}%)'.format(len(normal_answer), float(100*len(normal_answer)/len(df))))
normal_answer.head()

# RRRRRR -> RRLLLL にしてもanswer苦手なままの問題

normal_answer: 3468 (47.02%)


,answer_x,sp_x,answer_y,sp_y,answer,question,supporting_facts,context,type,old_sp_em,new_sp_em
5a83aaeb5542996488c2e483,Graduation,"[[Graduation (album), 0], [The College Dropout...",PRhyme,"[[The College Dropout, 0], [PRhyme, 0], [PRhym...",My Beautiful Dark Twisted Fantasy,What studio album did Kanye West record with R...,"[[Power (Kanye West song), 1], [My Beautiful D...","[[Hip Hop Since 1978, [Hip Hop Since 1978 (HHS...",bridge,0,0
5ae16ea85542990adbacf790,1975,"[[UVF Mid-Ulster Brigade, 5], [Miami Showband ...",1975,"[[The Miami Showband, 2], [Miami Showband kill...",31 July 1975,When did the UVF Mid-Ulster Brigade conducted ...,"[[UVF Mid-Ulster Brigade, 0], [UVF Mid-Ulster ...","[[The Miami Showband, [The Miami Showband were...",bridge,0,0
5ac10832554299012d1db687,22,"[[John Frink, 1], [The Ned-Liest Catch, 0], [5...",22,"[[John Frink, 1], [John Frink, 3], [500 Keys, ...",22 episodes.,How many episodes were in the 23rd season of a...,"[[500 Keys, 0], [500 Keys, 1], [500 Keys, 2], ...","[[John Frink, [John Frink (born May 5, 1964) i...",bridge,0,0
5ae2c4b8554299495565db31,Brookhaven,"[[Montgomery (village), New York, 0], [Brookha...",Rochester,"[[Glen Iris Inn, 0]]",Buffalo,What New York city is 60 miles northwest of th...,"[[Glen Iris Inn, 0], [Letchworth State Park, 2]]","[[Idylease Inn, [Idylease ( ""idle-ease""), a fo...",bridge,0,0
5a78d38455429970f5fffd94,Justin Smith,"[[Justin Smith (milliner), 0], [Justin Smith (...",Justin Smith,"[[Justin Smith (milliner), 1], [Amal Clooney, 0]]",Angelina Jolie,Who is one other client that is represented by...,"[[Justin Smith (milliner), 1], [Amal Clooney, 0]]","[[Shahram Taghavi, [Shahram Taghavi is a barri...",bridge,0,1


In [ ]:
normal_answer_good = deff_ans[deff_ans['old_em']==1].drop(columns='old_em')
normal_answer_good = normal_answer_good[normal_answer_good['new_em']==1].drop(columns='new_em')
print('normal_answer_good: {} ({:.2f}%)'.format(len(normal_answer_good), float(100*len(normal_answer_good)/len(df))))
normal_answer_good.head()

# RRRRRR -> RRLLLL 関係なくanswer得意な問題

normal_answer_good: 2429 (32.93%)


,answer_x,sp_x,answer_y,sp_y,answer,question,supporting_facts,context,type,old_sp_em,new_sp_em
5ae66d515542992ae0d16346,1927,"[[I'll Keep the Lovelight Burning, 0], [Patti ...",1927,"[[I'll Keep the Lovelight Burning, 0], [Patti ...",1927,"In what year was the singer who popularized ""I...","[[I'll Keep the Lovelight Burning, 0], [Patti ...","[[Don't Stop the Music (Robyn album), [Don't S...",bridge,1,1
5a8c66db554299585d9e3683,2017,"[[Eleventh Mom, 0], [Mom (film), 0], [Mom (fil...",2017,"[[Shashaa Tirupati, 1]]",2017,"What year was the film ""Mom"" released, starrin...","[[Shashaa Tirupati, 0], [Mom (film), 0]]","[[Garfield Gets Real, [Garfield Gets Real (als...",bridge,0,0
5add6b595542992200553b0c,"Atlanta, Georgia","[[Future (rapper), 1], [Shit (song), 0]]","Atlanta, Georgia","[[Future (rapper), 1], [Shit (song), 0]]","Atlanta, Georgia","Where was the artist behind ""Shit"" born and ra...","[[Shit (song), 0], [Future (rapper), 1]]","[[Sam Butcher, [Samuel ""Sam"" John Butcher (bor...",bridge,0,0
5ab29346554299545a2cf997,KXII,[],KXII,[],KXII,What CBS-affiliated station serves Pontotoc Co...,"[[KXII, 0], [Ada, Oklahoma, 0]]","[[Pontotoc County, Oklahoma, [Pontotoc County ...",bridge,0,0
5ab8179f5542990e739ec817,Dana Fox,"[[What Happens in Vegas, 0]]",Dana Fox,"[[Tom Vaughan (director), 1], [What Happens in...",Dana Fox,Who wrote Tom Vaughan's popular 2008 film?,"[[Tom Vaughan (director), 1], [What Happens in...","[[Tom Vaughan (director), [Tom Vaughan (born 5...",bridge,0,1


In [ ]:
print(better_answer['type'].value_counts())

bridge        755
comparison    166
Name: type, dtype: int64


In [ ]:
better_sp = deff_sp[deff_sp['old_sp_em']==0].drop(columns='old_sp_em')
better_sp = better_sp[better_sp['new_sp_em']==1].drop(columns='new_sp_em')
print('better_sp: {} ({:.2f}%)'.format(len(better_sp), float(100*len(better_sp)/len(df))))
better_sp.head()

# RRRRRR -> RRLLLL にすることでsp得意になった問題

better_sp: 503 (6.82%)


,answer_x,sp_x,answer_y,sp_y,answer,question,supporting_facts,context,type,old_em,new_em
5a78d38455429970f5fffd94,Justin Smith,"[[Justin Smith (milliner), 0], [Justin Smith (...",Justin Smith,"[[Justin Smith (milliner), 1], [Amal Clooney, 0]]",Angelina Jolie,Who is one other client that is represented by...,"[[Justin Smith (milliner), 1], [Amal Clooney, 0]]","[[Shahram Taghavi, [Shahram Taghavi is a barri...",bridge,0,0
5a8a38135542996c9b8d5e59,American socialite,[],Germany,"[[Ludwig Friedrich Leopold von Gerlach, 0], [O...",Prussia,Where did Otto von Bismarck and Ludwig Friedri...,"[[Ludwig Friedrich Leopold von Gerlach, 0], [O...","[[Bismarck 1862-1898, [Bismarck or Bismarck 18...",bridge,0,0
5ab8179f5542990e739ec817,Dana Fox,"[[What Happens in Vegas, 0]]",Dana Fox,"[[Tom Vaughan (director), 1], [What Happens in...",Dana Fox,Who wrote Tom Vaughan's popular 2008 film?,"[[Tom Vaughan (director), 1], [What Happens in...","[[Tom Vaughan (director), [Tom Vaughan (born 5...",bridge,1,1
5ab553e2554299494045ef81,Saturday Night Live,"[[Andy Spade, 0], [Andy Spade, 1], [A.M.A.N. (...",Saturday Night Live,"[[Andy Spade, 1], [David Spade, 1]]","""Saturday Night Live""","Which show was Andrew ""Andy"" Spade's brother i...","[[Andy Spade, 1], [David Spade, 1]]","[[Andy Taylor (The Andy Griffith Show), [Sheri...",bridge,0,0
5abbb42555429931dba144af,Jean-Paul Sartre,"[[Jean-Paul Sartre Experience, 0], [Jean-Paul ...",George Bernard Shaw,"[[Jean-Paul Sartre, 0], [George Bernard Shaw, 0]]",George Bernard Shaw,Jean-Paul Sartre or George Bernard Shaw have ...,"[[Jean-Paul Sartre, 0], [George Bernard Shaw, 0]]","[[Saint Genet, [Saint Genet, Actor and Martyr ...",comparison,0,1


In [ ]:
worse_sp = deff_sp[deff_sp['old_sp_em']==1].drop(columns='old_sp_em')
worse_sp = worse_sp[worse_sp['new_sp_em']==0].drop(columns='new_sp_em')
print('worse_sp: {} ({:.2f}%)'.format(len(worse_sp), float(100*len(worse_sp)/len(df))))
worse_sp.head()

# RRRRRR -> RRLLLL にしてsp苦手になった問題

worse_sp: 236 (3.20%)


,answer_x,sp_x,answer_y,sp_y,answer,question,supporting_facts,context,type,old_em,new_em
5a7f40de5542992e7d278cc8,"Richard ""Rick"" Ducommun","[[The 'Burbs, 0], [Rick Ducommun, 0]]",Carrie Fisher,"[[The 'Burbs, 0], [Rick Ducommun, 0], [Carrie ...","Richard ""Rick"" Ducommun",What star if the Burbs was Canadian?,"[[The 'Burbs, 0], [Rick Ducommun, 0]]","[[General Campaign Star, [The General Campaign...",bridge,1,0
5ae5e8dc554299546bf82fbe,Hakim Jamal,"[[Maulana Karenga, 1], [Hakim Jamal, 0]]",Hakim Jamal,"[[Maulana Karenga, 0], [Maulana Karenga, 1], [...",Hakim Abdullah Jamal,Which name adopted by African-American activis...,"[[Maulana Karenga, 1], [Hakim Jamal, 0]]","[[Etheridge Knight, [Etheridge Knight (April 1...",bridge,0,0
5ae4ec1e5542990ba0bbb19b,The Bears and I,"[[The Bears and I, 0], [The Bears and I, 2], [...",The Bears and I,"[[The Bears and I, 0], [The Bears and I, 2]]",The Bears and I,In between The Bears and I and Oceans which wa...,"[[The Bears and I, 0], [The Bears and I, 2], [...","[[The Bears and I, [The Bears and I is a 1974 ...",comparison,1,1
5a7fb17c5542994857a767bb,Detroit Lions and the Los Angeles Rams,"[[2007 San Diego State Aztecs football team, 0...",Detroit Lions and the Los Angeles Rams,"[[2007 San Diego State Aztecs football team, 1...",Detroit Lions and the Los Angeles Rams,Which two teams did the the head coach of The ...,"[[2007 San Diego State Aztecs football team, 0...","[[San Diego State Aztecs baseball, [The San Di...",bridge,1,1
5adff4a555429925eb1afb91,Texarkana Regional Airport,"[[Wilmington International Airport, 0], [Texar...",Texarkana Regional Airport,"[[Wilmington International Airport, 0], [Texar...",Wilmington International Airport,Which airport is closer to the Atlantic ocean:...,"[[Wilmington International Airport, 0], [Texar...","[[Wilmington International Airport, [Wilmingto...",comparison,0,0


In [ ]:
normal_sp = deff_sp[deff_sp['old_sp_em']==0].drop(columns='old_sp_em')
normal_sp = normal_sp[normal_sp['new_sp_em']==0].drop(columns='new_sp_em')
print('better_sp: {} ({:.2f}%)'.format(len(normal_sp), float(100*len(normal_sp)/len(df))))
normal_sp.head()

# RRRRRR -> RRLLLL にしてもsp苦手なままの問題

better_sp: 6208 (84.16%)


,answer_x,sp_x,answer_y,sp_y,answer,question,supporting_facts,context,type,old_em,new_em
5a83aaeb5542996488c2e483,Graduation,"[[Graduation (album), 0], [The College Dropout...",PRhyme,"[[The College Dropout, 0], [PRhyme, 0], [PRhym...",My Beautiful Dark Twisted Fantasy,What studio album did Kanye West record with R...,"[[Power (Kanye West song), 1], [My Beautiful D...","[[Hip Hop Since 1978, [Hip Hop Since 1978 (HHS...",bridge,0,0
5a8c66db554299585d9e3683,2017,"[[Eleventh Mom, 0], [Mom (film), 0], [Mom (fil...",2017,"[[Shashaa Tirupati, 1]]",2017,"What year was the film ""Mom"" released, starrin...","[[Shashaa Tirupati, 0], [Mom (film), 0]]","[[Garfield Gets Real, [Garfield Gets Real (als...",bridge,1,1
5ae16ea85542990adbacf790,1975,"[[UVF Mid-Ulster Brigade, 5], [Miami Showband ...",1975,"[[The Miami Showband, 2], [Miami Showband kill...",31 July 1975,When did the UVF Mid-Ulster Brigade conducted ...,"[[UVF Mid-Ulster Brigade, 0], [UVF Mid-Ulster ...","[[The Miami Showband, [The Miami Showband were...",bridge,0,0
5ac10832554299012d1db687,22,"[[John Frink, 1], [The Ned-Liest Catch, 0], [5...",22,"[[John Frink, 1], [John Frink, 3], [500 Keys, ...",22 episodes.,How many episodes were in the 23rd season of a...,"[[500 Keys, 0], [500 Keys, 1], [500 Keys, 2], ...","[[John Frink, [John Frink (born May 5, 1964) i...",bridge,0,0
5ae2c4b8554299495565db31,Brookhaven,"[[Montgomery (village), New York, 0], [Brookha...",Rochester,"[[Glen Iris Inn, 0]]",Buffalo,What New York city is 60 miles northwest of th...,"[[Glen Iris Inn, 0], [Letchworth State Park, 2]]","[[Idylease Inn, [Idylease ( ""idle-ease""), a fo...",bridge,0,0


In [ ]:
normal_sp_good = deff_sp[deff_sp['old_sp_em']==1].drop(columns='old_sp_em')
normal_sp_good = normal_sp_good[normal_sp_good['new_sp_em']==1].drop(columns='new_sp_em')
print('normal_sp_good: {} ({:.2f}%)'.format(len(normal_sp_good), float(100*len(normal_sp_good)/len(df))))
normal_sp_good.head()

# RRRRRR -> RRLLLL 関係なくもとから得意な問題

normal_sp_good: 429 (5.82%)


,answer_x,sp_x,answer_y,sp_y,answer,question,supporting_facts,context,type,old_em,new_em
5ae66d515542992ae0d16346,1927,"[[I'll Keep the Lovelight Burning, 0], [Patti ...",1927,"[[I'll Keep the Lovelight Burning, 0], [Patti ...",1927,"In what year was the singer who popularized ""I...","[[I'll Keep the Lovelight Burning, 0], [Patti ...","[[Don't Stop the Music (Robyn album), [Don't S...",bridge,1,1
5adcceda5542990d50227d31,Brazil,"[[Judith Weir, 0], [George Gershwin, 0]]",British,"[[Judith Weir, 0], [George Gershwin, 0]]",a British composer,George Gershwin is an American Composer and Ju...,"[[Judith Weir, 0], [George Gershwin, 0]]","[[Jamie Brown (composer), [Jamie Brown (born 1...",comparison,0,0
5abb8afe5542993f40c73b1e,no,"[[Roger Donaldson, 0], [André Cayatte, 0]]",no,"[[Roger Donaldson, 0], [André Cayatte, 0]]",no,Were both Roger Donaldson and André Cayatte fr...,"[[Roger Donaldson, 0], [André Cayatte, 0]]","[[Roger Donaldson, [Roger Donaldson (born 15 N...",comparison,1,1
5ae3d2715542990afbd1e1e9,yes,"[[Control Room (film), 0], [Meeting Resistance...",yes,"[[Control Room (film), 0], [Meeting Resistance...",yes,Are Control Room and Meeting Resistance both d...,"[[Control Room (film), 0], [Meeting Resistance...","[[Control Room (film), [Control Room is a 2004...",comparison,1,1
5ab925c755429919ba4e23d6,screenwriter,"[[J. Searle Dawley, 0], [Ken Annakin, 0]]",film director,"[[J. Searle Dawley, 0], [Ken Annakin, 0]]","Kenneth Cooper ""Ken"" Annakin, OBE (10 August 1...",J. Searle Dawley and Ken Annakin were both dir...,"[[J. Searle Dawley, 0], [Ken Annakin, 0]]","[[J. Searle Dawley, [James Searle Dawley (May ...",comparison,0,0


In [ ]:
same_better = pd.merge(better_answer, better_sp.drop(columns='answer'), left_index=True, right_index=True)
print(' - answer･sp共に解けるようになった問題: {}'.format(len(same_better)))

 - answer･sp共に解けるようになった問題: 69


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ans_li = [len(better_answer), float(100*len(better_answer)/len(df)), len(worse_answer), float(100*len(worse_answer)/len(df)), len(normal_answer_good), float(100*len(normal_answer_good)/len(df)), len(normal_answer), float(100*len(normal_answer)/len(df))]
sp_li  = [len(better_sp),     float(100*len(better_sp)/len(df)),     len(worse_sp),     float(100*len(worse_sp)/len(df)),     len(normal_sp_good),  float(100*len(normal_sp_good) /len(df)), len(normal_sp),     float(100*len(normal_sp)/len(df))]
print('answer:\n - 得意になった...{}({:.2f}%)\n - 苦手になった...{}({:.2f}%)\n - もとから得意...{}({:.2f}%)\n - もとから苦手...{}({:.2f}%)'.format(ans_li[0], ans_li[1], ans_li[2], ans_li[3], ans_li[4], ans_li[5], ans_li[6], ans_li[7]))
print('合計...{}({:.2f}%)'.format(ans_li[0]+ans_li[2]+ans_li[4]+ans_li[6], ans_li[1]+ans_li[3]+ans_li[5]+ans_li[7]))
print('sp:\n - 得意になった...{}({:.2f}%)\n - 苦手になった...{}({:.2f}%)\n - もとから得意...{}({:.2f}%)\n - もとから苦手...{}({:.2f}%)'.format(sp_li[0], sp_li[1], sp_li[2], sp_li[3], sp_li[4], sp_li[5], sp_li[6], sp_li[7]))
print('合計...{}({:.2f}%)'.format(sp_li[0]+sp_li[2]+sp_li[4]+sp_li[6], sp_li[1]+sp_li[3]+sp_li[5]+sp_li[7]))

answer:
 - 得意になった...921(12.49%)
 - 苦手になった...558(7.57%)
 - もとから得意...2429(32.93%)
 - もとから苦手...3468(47.02%)
合計...7376(100.00%)
sp:
 - 得意になった...503(6.82%)
 - 苦手になった...236(3.20%)
 - もとから得意...429(5.82%)
 - もとから苦手...6208(84.16%)
合計...7376(100.00%)


LLLLLL
 - 得意になった...567(7.69%)
 - 苦手になった...674(9.14%)
 - もとから得意...2313(31.36%)
 - もとから苦手...3822(51.82%)  
合計...7376(100.00%)

RRLLLL
 - 得意になった...921(12.49%)
 - 苦手になった...558(7.57%)
 - もとから得意...2429(32.93%)
 - もとから苦手...3468(47.02%)  
合計...7376(100.00%)

In [ ]:
!pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from googletrans import Translator
import tqdm
import numpy as np
import time
def writer(samples, output='file', file_name='sample.txt', file_title: str=None, translate=True, print_true_ans=False):
  print('output mode      : {}'.format(output))
  print('file_name        : {}'.format(file_name))
  print('file_title       : {}'.format(file_title))
  print('translate        : {}'.format(translate))
  print('print_true_answer: {}'.format(print_true_ans))
  path = '/content/drive/MyDrive/GradReserch/hotpot-master/kaiseki/' + file_name
  i = 1
  lines = []
  for row in samples.itertuples():
    lines.append('[{}]'.format(i))
    lines.append('\n')
    lines.append('context: ')
    lines.append('\n')
    for j in range(len(row.context)):
      k = 1
      for texts in row.context[j]:
        if k==1:
          lines.append('\"{}\"'.format(texts))
        else:
          for text in texts:
            lines.append('{}'.format(text.strip()))
          lines.append('')
        k+=1
    lines.append('question:')
    lines.append('{}'.format(row.question))
    lines.append('\n')
    lines.append('answer(1):')
    lines.append('{}'.format(row.answer_x))
    lines.append('supporting facts:')
    for one in row.sp_x:
      lines.append('{} {}'.format(one[0], one[1]))
    lines.append('\n')
    lines.append('answer(2):')
    lines.append('{}'.format(row.answer_y))
    lines.append('supporting facts:')
    for one in row.sp_y:
      lines.append('{} {}'.format(one[0], one[1]))
    lines.append('\n')
    if print_true_ans==True:
      lines.append('answer(true):')
      lines.append('{}'.format(row.answer))
      lines.append('supporting facts:')
      for one in row.supporting_facts:
        lines.append('{} {}'.format(one[0], one[1]))
      lines.append('\n')
    lines.append('\n')
    i+=1

  if translate == True:
    sleep_num = 25 
    tl = Translator()
    translated = []
    for i in tqdm.tqdm(range(len(lines))):
      if i%sleep_num == 0:  # Google translate の翻訳量制限対策
        time.sleep(0.1)
      if (lines[i] != '') & (lines[i] != '\n') & (i != 0):
        translated.append(tl.translate(text=lines[i], src='en', dest='ja').text)
      elif i == 0:
        translated.append(lines[i])
      else:
        translated.append('')
    lines = translated
  
  if output == 'file':
    f = open(path, 'w', encoding='UTF-8')
    if file_title != None:
      f.write('[{}]\n\n'.format(file_title))
    for line in lines:
      f.write('{}\n'.format(line))
    f.close()

  elif output == 'print':
    print('')
    for line in lines:
      print('   {}'.format(line))

### ショートカット（その2）

#### 前準備

モデルA・B・Cによる「回答・根拠」の正解/不正解を1/0として、  
次の変数を用意している。
- すべてまとめたもの **df**
- 解けたモデルによる分類 **a000** 〜 **a111** （aABCの順で、解ける:1 解けない:0 としている）

In [ ]:
import pandas as pd
import numpy as np

modelA_path = '/content/drive/MyDrive/GradReserch/hotpot-master/kaiseki/data/dev_distractor_pred_RRRRRR.json'
modelB_path = '/content/drive/MyDrive/GradReserch/hotpot-master/kaiseki/data/dev_distractor_pred_LLLLLL.json'
modelC_path = '/content/drive/MyDrive/GradReserch/hotpot-master/kaiseki/data/dev_distractor_pred_RRLLLL.json'
gold_path   = '/content/drive/MyDrive/GradReserch/hotpot-master/kaiseki/data/hotpot_dev_distractor_v1.json'

all    = pd.read_json(gold_path).set_index('_id')
gold   = pd.read_json(gold_path).set_index('_id').drop(columns=['question', 'type', 'context', 'level'])
modelA = pd.read_json(modelA_path)
modelB = pd.read_json(modelB_path)
modelC = pd.read_json(modelC_path)
modelA = pd.merge(modelA, gold, left_index=True, right_index=True)
modelB = pd.merge(modelB, gold, left_index=True, right_index=True)
modelC = pd.merge(modelC, gold, left_index=True, right_index=True)

li_add = ['question', 'type', 'level']
add    = pd.read_json(gold_path).set_index('_id')
for column in add.columns:
  if column not in li_add:
    add = add.drop(columns=column)

In [ ]:
choices = [1]

conditions = [
    modelA['answer_x'] == modelA['answer_y']
]
modelA['ansA'] = np.select(conditions, choices, default=0)

conditions = [
    modelA['sp'] == modelA['supporting_facts']
]
modelA['spA'] = np.select(conditions, choices, default=0)

modelA.drop(columns=['answer_x', 'answer_y', 'sp', 'supporting_facts'], inplace=True)

conditions = [
    modelB['answer_x'] == modelB['answer_y']
]
modelB['ansB'] = np.select(conditions, choices, default=0)

conditions = [
    modelB['sp'] == modelB['supporting_facts']
]
modelB['spB'] = np.select(conditions, choices, default=0)

modelB.drop(columns=['answer_x', 'answer_y', 'sp', 'supporting_facts'], inplace=True)

conditions = [
    modelC['answer_x'] == modelC['answer_y']
]
modelC['ansC'] = np.select(conditions, choices, default=0)

conditions = [
    modelC['sp'] == modelC['supporting_facts']
]
modelC['spC'] = np.select(conditions, choices, default=0)

modelC.drop(columns=['answer_x', 'answer_y', 'sp', 'supporting_facts'], inplace=True)

In [ ]:
df = pd.merge(modelA, modelB, right_index=True, left_index=True)
df = pd.merge(df, modelC, right_index=True, left_index=True)
df = pd.merge(df, add, right_index=True, left_index=True)

In [ ]:
ans000 = df.query('(ansA==0) & (ansB==0) & (ansC==0)')
ans111 = df.query('(ansA==1) & (ansB==1) & (ansC==1)')

ans001 = df.query('(ansA==0) & (ansB==0) & (ansC==1)')
ans010 = df.query('(ansA==0) & (ansB==1) & (ansC==0)')
ans100 = df.query('(ansA==1) & (ansB==0) & (ansC==0)')

ans011 = df.query('(ansA==0) & (ansB==1) & (ansC==1)')
ans101 = df.query('(ansA==1) & (ansB==0) & (ansC==1)')
ans110 = df.query('(ansA==1) & (ansB==1) & (ansC==0)')

In [ ]:
ans100.head(3)

,ansA,spA,ansB,spB,ansC,spC,question,type,level
5a7f40de5542992e7d278cc8,1,1,0,0,0,0,What star if the Burbs was Canadian?,bridge,hard
5adbf84555429947ff17387c,1,0,0,0,0,0,Are both Adolfo Bioy Casares and James Norman ...,comparison,hard
5a889d18554299206df2b310,1,1,0,0,0,1,What kind of group does Pinhead Gunpowder and ...,bridge,hard


In [ ]:
ans010.head(3)

,ansA,spA,ansB,spB,ansC,spC,question,type,level
5a83aaeb5542996488c2e483,0,0,1,0,0,0,What studio album did Kanye West record with R...,bridge,hard
5a7c38c955429935c91b5132,0,0,1,0,0,0,What characteristic meaning it's transferable ...,bridge,hard
5a9087ac5542990a98493668,0,0,1,0,0,0,"How many movies did the ""Comic Book Villains"" ...",bridge,hard


In [ ]:
ans001.head(3)

,ansA,spA,ansB,spB,ansC,spC,question,type,level
5a8790785542993e715abfac,0,0,0,0,1,0,What was the first name of the wife of the nam...,bridge,hard
5adfc97255429906c02daa41,0,0,0,0,1,0,In which tier of the English football league s...,bridge,hard
5abbb42555429931dba144af,0,0,0,0,1,1,Jean-Paul Sartre or George Bernard Shaw have ...,comparison,hard


#### 質問文の自動翻訳・出力機

In [ ]:
!pip install googletrans==4.0.0-rc1

In [ ]:
from googletrans import Translator
import tqdm
import numpy as np
import time
def export_ques(df, filename='sample.txt', translate=True):
  path = '/content/drive/MyDrive/GradReserch/hotpot-master/kaiseki/kind/' + filename

  if translate == True:
    lines = []
    tl = Translator()
    i = 0
    for line in tqdm.tqdm(df['question']):
      time.sleep(5)
      lines.append(tl.translate(text=line, src='en', dest='ja').text)
    print('[translate] process has done.')

  else:
    lines = df['question']

  f = open(path, 'w', encoding='UTF-8')
  f.write('[{}]\n\n'.format(filename.replace('.txt', '')))
  for line in tqdm.tqdm(lines):
    f.write('{}\n'.format(line))
  print('[export] process has done.')
  f.close()

In [ ]:
export_ques(ans000, filename='[en]anyone_couldnot_ans.txt', translate=False)
export_ques(ans111, filename='[en]everyone_could_ans.txt', translate=False)
export_ques(ans100, filename='[en]onlyA_could_ans.txt', translate=False)
export_ques(ans010, filename='[en]onlyB_could_ans.txt', translate=False)
export_ques(ans001, filename='[en]onlyC_could_ans.txt', translate=False)
export_ques(ans011, filename='[en]onlyA_couldnot_ans.txt', translate=False)
export_ques(ans101, filename='[en]onlyB_couldnot_ans.txt', translate=False)
export_ques(ans110, filename='[en]onlyC_couldnot_ans.txt', translate=False)

#### 分析１（列要素カウンタ）

In [ ]:
print(len(df))
_1xx = len(ans100) + len(ans101) + len(ans110) + len(ans111)
_x1x = len(ans010) + len(ans011) + len(ans110) + len(ans111)
_xx1 = len(ans001) + len(ans011) + len(ans101) + len(ans111)
print('{} {}'.format(_1xx, len(df)-_1xx))
print('{} {}'.format(_x1x, len(df)-_x1x))
print('{} {}'.format(_xx1, len(df)-_xx1))

7376
2987 4389
2880 4496
3350 4026


In [ ]:
def kind_count(column):
  ques_info = []
  li_temp = []
  '''
  li_temp.append(ans000)
  li_temp.append(ans001)
  li_temp.append(ans010)
  li_temp.append(ans011)
  li_temp.append(ans100)
  li_temp.append(ans101)
  li_temp.append(ans110)
  li_temp.append(ans111)
  ans1xx = pd.concat([ans100, ans101, ans110, ans111], axis=0, ignore_index=True)
  ansx1x = pd.concat([ans010, ans011, ans110, ans111], axis=0, ignore_index=True)
  ansxx1 = pd.concat([ans001, ans011, ans101, ans111], axis=0, ignore_index=True)
  li_temp.append(ans1xx)
  li_temp.append(ansx1x)
  li_temp.append(ansxx1)
  '''

  li_temp.append(df)
  for a_set in li_temp:
    ques_type, count = [], []
    for i, row in a_set.drop(columns=['spA', 'spB', 'spC']).iterrows():
      type_is = False
      for j, types in enumerate(ques_type):
        if row[column] == types:
          type_is = True
          count[j] += 1
      if type_is == False:
        ques_type.append(row[column])
        count.append(1)
    ques_info.append((ques_type, count))
  return ques_info

In [ ]:
def kind_print(column, index=None):
  ques_info = kind_count(column)
  if index == None:
    index = range(len(ques_info))
  for i in index:
    ques_type, count = ques_info[i]
    print(' [{}]'.format(i))
    sum = 0
    for j in range(len(count)):
      sum += count[j]
    for j in range(len(ques_type)):
      print('  -> {:4d} ({:4.1f}%, 全体の{:.1f}%) ... {}'.format(count[j], 100*count[j]/sum, 100*count[j]/len(df), ques_type[j]))


In [ ]:
# index = [4, 2, 1]
index=None

print('各モデルの解けた問題における，推論タイプ')
kind_print('type', index=index)

print('\n各モデルの解けた問題における、推論難易度')
kind_print('level', index=index)

各モデルの解けた問題における，推論タイプ
 [0]
  -> 5891 (79.9%, 全体の79.9%) ... bridge
  -> 1485 (20.1%, 全体の20.1%) ... comparison

各モデルの解けた問題における、推論難易度
 [0]
  -> 7376 (100.0%, 全体の100.0%) ... hard


#### 分析２（その他）

In [ ]:
li = []
li.append(len(ans000))
li.append(len(ans111))
li.append(len(ans001))
li.append(len(ans010))
li.append(len(ans100))
li.append(len(ans011))
li.append(len(ans101))
li.append(len(ans110))
idx = ['ans000', 'ans111', 'ans001', 'ans010', 'ans100', 'ans011', 'ans101', 'ans110']
li_df = pd.Series(li, index=idx)

all = 0
for line in li:
  all += line

for i in range(len(li_df)):
  per = li_df[i] * 100 / all
  print('{}: {:.2f}%'.format(idx[i], per))

per = (li_df[2] + li_df[4] + li_df[6]) * 100 / all
print('{:.2f}%'.format(per))

per = (li_df[3] + li_df[5] + li_df[7]) * 100 / all
print('{:.2f}%'.format(per))

ans000: 43.38%
ans111: 27.70%
ans001: 3.63%
ans010: 8.43%
ans100: 3.90%
ans011: 4.05%
ans101: 3.66%
ans110: 5.23%
11.20%
17.72%


In [ ]:
df.head()

,ansA,spA,ansB,spB,ansC,spC,question,type
5ae66d515542992ae0d16346,1,1,1,1,1,0,"In what year was the singer who popularized ""I...",bridge
5a83aaeb5542996488c2e483,0,0,0,0,1,0,What studio album did Kanye West record with R...,bridge
5a8c66db554299585d9e3683,1,0,1,0,0,0,"What year was the film ""Mom"" released, starrin...",bridge
5ae16ea85542990adbacf790,0,0,0,0,0,0,When did the UVF Mid-Ulster Brigade conducted ...,bridge
5ac10832554299012d1db687,0,0,0,0,0,0,How many episodes were in the 23rd season of a...,bridge


In [ ]:
ques_type, count = [], []
for i, row in df.iterrows():
  type_is = False
  for j, types in enumerate(ques_type):
    if row['type'] == types:
      type_is = True
      count[j] += 1
  if type_is == False:
    ques_type.append(row['type'])
    count.append(1)

print('全ての問題の，推論タイプ\n')
sum = 0
for i in range(len(count)):
  sum += count[i]
for i in range(len(ques_type)):
  print('{} ({:.1f}%) ... {}'.format(count[i], 100*count[i]/sum, ques_type[i]))
print('----------------------------')
print('{} ({:.1f}%) ... 合計'.format(sum, 100.0*sum/sum))

全ての問題の，推論タイプ

5891 (79.9%) ... bridge
1485 (20.1%) ... comparison
----------------------------
7376 (100.0%) ... 合計


In [ ]:
print(len(df))

7376


In [ ]:
a000 = ans000['question'].values.tolist()
a001 = ans001['question'].values.tolist()
a010 = ans010['question'].values.tolist()
a011 = ans011['question'].values.tolist()
a100 = ans100['question'].values.tolist()
a101 = ans101['question'].values.tolist()
a110 = ans110['question'].values.tolist()
a111 = ans111['question'].values.tolist()

a000 = [x.split() for x in a000]
a001 = [x.split() for x in a001]
a010 = [x.split() for x in a010]
a011 = [x.split() for x in a011]
a100 = [x.split() for x in a100]
a101 = [x.split() for x in a101]
a110 = [x.split() for x in a110]
a111 = [x.split() for x in a111]

li = []
li.append(a000)
li.append(a001)
li.append(a010)
li.append(a011)
li.append(a100)
li.append(a101)
li.append(a110)
for i in range(len(li)):
  # print('[{}] {}'.format(i, li[i][0]))
  print('[{}]'.format(i))
  for j in range(0, 5, 1):
    print('  {}'.format(li[i][j]))

[0]
  ['When', 'did', 'the', 'UVF', 'Mid-Ulster', 'Brigade', 'conducted', 'the', 'attack', 'The', 'Miami', 'Showband', 'killings?']
  ['How', 'many', 'episodes', 'were', 'in', 'the', '23rd', 'season', 'of', 'a', 'Fox', 'network', 'show', 'which,', 'in', "it's", 'twenty-second', 'season,', 'aired', "it's", 'twenty-first', 'episode', 'which', 'was', 'written', 'by', 'John', 'Frink', 'and', 'directed', 'by', 'Bob', 'Anderson.']
  ['What', 'New', 'York', 'city', 'is', '60', 'miles', 'northwest', 'of', 'the', 'Glen', 'Iris', 'Inn?']
  ['Who', 'is', 'one', 'other', 'client', 'that', 'is', 'represented', 'by', 'the', 'same', 'person', 'that', 'also', 'represents', 'a', 'barrister', 'at', 'Doughty', 'Street', 'Chambers?']
  ['What', 'program', 'responsible', 'for', 'the', 'Health', 'Insurance', 'Portability', 'and', 'Accountability', 'Act', 'works', 'with', 'AHCCS?']
[1]
  ['What', 'studio', 'album', 'did', 'Kanye', 'West', 'record', 'with', 'Roc-A-Fella', 'Records', 'and', 'soul', 'singer', '

In [ ]:
import nltk
ps = nltk.stem.porter.PorterStemmer()

li_a = li
words = []
for i in range(len(li_a)):
  temp = []
  for j in range(len(li_a[i])):
    li_a[i][j] = [ps.stem(word) for word in li_a[i][j]]
    for word in li_a[i][j]:
      temp.append(word)
  print('[{}]'.format(i))
  for j in range(0, 3, 1):
    print('  {}'.format(li_a[i][j]))
  words.append(temp)

print(words[0][:30])

[0]
  ['when', 'did', 'the', 'uvf', 'mid-ulst', 'brigad', 'conduct', 'the', 'attack', 'the', 'miami', 'showband', 'killings?']
  ['how', 'mani', 'episod', 'were', 'in', 'the', '23rd', 'season', 'of', 'a', 'fox', 'network', 'show', 'which,', 'in', "it'", 'twenty-second', 'season,', 'air', "it'", 'twenty-first', 'episod', 'which', 'wa', 'written', 'by', 'john', 'frink', 'and', 'direct', 'by', 'bob', 'anderson.']
  ['what', 'new', 'york', 'citi', 'is', '60', 'mile', 'northwest', 'of', 'the', 'glen', 'iri', 'inn?']
[1]
  ['what', 'studio', 'album', 'did', 'kany', 'west', 'record', 'with', 'roc-a-fella', 'record', 'and', 'soul', 'singer', 'dwele?']
  ['what', 'characterist', 'mean', "it'", 'transfer', 'between', 'species,', 'doe', 'monkeypox', 'and', 'cowpox', 'viru', 'both', 'share?']
  ['how', 'mani', 'movi', 'did', 'the', '"comic', 'book', 'villains"', 'star', 'born', 'on', 'march', '20,1970', 'apprear', 'in', 'sinc', 'the', 'earli', '1990s?']
[2]
  ['what', 'wa', 'the', 'first', 'name',

In [ ]:
from collections import Counter
n = 50
results = []
keyword, count, pack = [], [], [[]]
for i, kind in enumerate(words):
  fdist = Counter(kind)
  result = fdist.most_common(n=n)
  results.append(result)
  if i in [1, 2, 4]:
    # print('[{}]'.format(i))
    for k, data in enumerate(result):
      # print('  ({:2d}) {}'.format(k+1, data))
      word_is = False
      for l, word in enumerate(keyword):
        if word == data[0]:
          count[l] += 1
          pack[l].append(i)
          word_is = True
      if word_is == False:
        keyword.append(data[0])
        count.append(1)
        pack.append([i])

for i in range(0, n, 1):
  if count[i] <= 1:
    print('({}: {}) {}'.format(count[i], pack[i], keyword[i]))

(1: [1]) ?
(1: [1]) new
(1: [1]) role


In [ ]:
import nltk
nltk.download('stopwords')
lst_stopwords = nltk.corpus.stopwords.words('english')
lst_stopwords[:20]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his']

In [ ]:
li_a = li
for i in range(len(li_a)):
  for j in range(len(li_a[i])):
    li_a[i][j] = [word for word in li_a[i][j] if word not in lst_stopwords]
  print('[{}]'.format(i))
  for j in range(0, 3, 1):
    print('  {}'.format(li_a[i][j]))

[0]
  ['uvf', 'mid-ulst', 'brigad', 'conduct', 'attack', 'miami', 'showband', 'killings?']
  ['mani', 'episod', '23rd', 'season', 'fox', 'network', 'show', 'which,', "it'", 'twenty-second', 'season,', 'air', "it'", 'twenty-first', 'episod', 'wa', 'written', 'john', 'frink', 'direct', 'bob', 'anderson.']
  ['new', 'york', 'citi', '60', 'mile', 'northwest', 'glen', 'iri', 'inn?']
[1]
  ['studio', 'album', 'kani', 'west', 'record', 'roc-a-fella', 'record', 'soul', 'singer', 'dwele?']
  ['characterist', 'mean', "it'", 'transfer', 'species,', 'doe', 'monkeypox', 'cowpox', 'viru', 'share?']
  ['mani', 'movi', '"comic', 'book', 'villains"', 'star', 'born', 'march', '20,1970', 'apprear', 'sinc', 'earli', '1990s?']
[2]
  ['wa', 'first', 'name', 'wife', 'namesak', 'high', 'school', 'locat', '123', 'cole', 'street', 'jersey', 'city?']
  ['tier', 'english', 'footbal', 'leagu', 'system', 'doe', 'team', 'compet', 'nott', 'counti', '1894', 'fa', 'cup', 'final', 'current', 'compete?']
  ['jean-paul', 

In [ ]:
ps = nltk.stem.porter.PorterStemmer()

words = []
for i in range(len(li_a)):
  temp = []
  for j in range(len(li_a[i])):
    li_a[i][j] = [ps.stem(word) for word in li_a[i][j]]
    for word in li_a[i][j]:
      temp.append(word)
  print('[{}]'.format(i))
  for j in range(0, 3, 1):
    print('  {}'.format(li_a[i][j]))
  words.append(temp)

[0]
  ['uvf', 'mid-ulst', 'brigad', 'conduct', 'attack', 'miami', 'showband', 'killings?']
  ['mani', 'episod', '23rd', 'season', 'fox', 'network', 'show', 'which,', "it'", 'twenty-second', 'season,', 'air', "it'", 'twenty-first', 'episod', 'wa', 'written', 'john', 'frink', 'direct', 'bob', 'anderson.']
  ['new', 'york', 'citi', '60', 'mile', 'northwest', 'glen', 'iri', 'inn?']
[1]
  ['studio', 'album', 'kani', 'west', 'record', 'roc-a-fella', 'record', 'soul', 'singer', 'dwele?']
  ['characterist', 'mean', "it'", 'transfer', 'species,', 'doe', 'monkeypox', 'cowpox', 'viru', 'share?']
  ['mani', 'movi', '"comic', 'book', 'villains"', 'star', 'born', 'march', '20,1970', 'apprear', 'sinc', 'earli', '1990s?']
[2]
  ['wa', 'first', 'name', 'wife', 'namesak', 'high', 'school', 'locat', '123', 'cole', 'street', 'jersey', 'city?']
  ['tier', 'english', 'footbal', 'leagu', 'system', 'doe', 'team', 'compet', 'nott', 'counti', '1894', 'fa', 'cup', 'final', 'current', 'compete?']
  ['jean-paul', 

In [ ]:
from collections import Counter
results = []
for i, kind in enumerate(words):
  fdist = Counter(kind)
  result = fdist.most_common(n=10)
  if i in [1, 2, 4]:
    print('[{}] {}'.format(i, result))
  results.append(result)

[1] [('wa', 98), ('film', 26), ('ha', 22), ('american', 21), ('doe', 16), ('born', 16), ('first,', 16), ('first', 13), ('mani', 12), ('star', 12)]
[2] [('wa', 204), ('film', 54), ('american', 49), ('play', 45), ('star', 43), ('name', 38), ('born', 35), ('ha', 33), ('also', 29), ('known', 27)]
[4] [('wa', 84), ('film', 26), ('name', 25), ('american', 18), ('star', 17), ('ha', 14), ('first', 12), ('born', 12), ('album', 12), ('former', 12)]


In [ ]:
lem = nltk.stem.wordnet.WordNetLemmatizer()
nltk.download('omw-1.4')

for i in range(len(li_a)):
  for j in range(len(li_a[i])):
    li_a[i][j] = [lem.lemmatize(word) for word in li_a[i][j]]
  print('[{}]'.format(i))
  for j in range(0, 3, 1):
    print('  {}'.format(li_a[i][j]))

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


[0]
  ['when', 'uvf', 'mid-ulst', 'brigad', 'conduct', 'attack', 'the', 'miami', 'showband', 'killings?']
  ['how', 'mani', 'episod', '23rd', 'season', 'fox', 'network', 'show', 'which,', 'twenty-second', 'season,', 'air', 'twenty-first', 'episod', 'written', 'john', 'frink', 'direct', 'bob', 'anderson.']
  ['what', 'new', 'york', 'citi', '60', 'mile', 'northwest', 'glen', 'iri', 'inn?']
[1]
  ['what', 'studio', 'album', 'kany', 'west', 'record', 'roc-a-fella', 'record', 'soul', 'singer', 'dwele?']
  ['what', 'characterist', 'mean', 'transfer', 'species,', 'monkeypox', 'cowpox', 'viru', 'share?']
  ['how', 'mani', 'movi', '"comic', 'book', 'villains"', 'star', 'born', 'march', '20,1970', 'apprear', 'sinc', 'earli', '1990s?']
[2]
  ['what', 'first', 'name', 'wife', 'namesak', 'high', 'school', 'locat', '123', 'cole', 'street', 'jersey', 'city?']
  ['in', 'tier', 'english', 'footbal', 'leagu', 'system', 'team', 'compet', 'nott', 'counti', '1894', 'fa', 'cup', 'final', 'current', 'compete

In [ ]:
li_df = []
li_df.append(ans000)
li_df.append(ans001)
li_df.append(ans010)
li_df.append(ans011)
li_df.append(ans100)
li_df.append(ans101)
li_df.append(ans110)
li_df.append(ans111)

comment = [
    '全モデルが苦手とする問題',
    'モデルCのみが得意とする問題',
    'モデルBのみが得意とする問題',
    'モデルB･Cが得意とする問題',
    'モデルAのみが得意とする問題',
    'モデルA･Cが得意とする問題',
    'モデルA･Bが得意とする問題',
    '全モデルが得意とする問題'
]
for i in range(len(li_df)):
  li_df[i].loc[:, 'word_count'] = li_df[i]['question'].apply(lambda x: len(str(x).split(' ')))
  li_df[i].loc[:, 'char_count'] = li_df[i]['question'].apply(lambda x: sum(len(word) for word in str(x).split(' ')))
  li_df[i].loc[:, 'avg_word_length'] = li_df[i]['char_count'] / li_df[i]['word_count']
  li_df[i].loc[:, 'avg_sentence_lenght'] = li_df[i]['word_count']

for i in [0, 7, 4, 6, 2, 3, 1, 5]:
  print('[{}]: {}'.format(i, comment[i]))
  print('  {}'.format(li_df[i]['question'][0]))
  display(li_df[i].drop(columns=['spA', 'spB', 'spC']).head(2))
  print()

temp = li_df
print('temp')
for i in range(len(temp)):
  temp[i] = temp[i].drop(columns=['ansA', 'ansB', 'ansC', 'spA', 'spB', 'spC', 'question', 'sentence_count'])
print(temp[0].columns)

[0]: 全モデルが苦手とする問題
  When did the UVF Mid-Ulster Brigade conducted the attack The Miami Showband killings?


,ansA,ansB,ansC,question,word_count,char_count,sentence_count,avg_word_length,avg_sentence_lenght
5ae16ea85542990adbacf790,0,0,0,When did the UVF Mid-Ulster Brigade conducted ...,13,73,1,5.615385,13
5ac10832554299012d1db687,0,0,0,How many episodes were in the 23rd season of a...,33,162,2,4.909091,33



[7]: 全モデルが得意とする問題
  In what year was the singer who popularized "I'll Keep the Lovelight Burning" born?


,ansA,ansB,ansC,question,word_count,char_count,sentence_count,avg_word_length,avg_sentence_lenght
5ae66d515542992ae0d16346,1,1,1,"In what year was the singer who popularized ""I...",14,70,1,5.000000,14
5add6b595542992200553b0c,1,1,1,"Where was the artist behind ""Shit"" born and ra...",9,43,1,4.777778,9



[4]: モデルAのみが得意とする問題
  What star if the Burbs was Canadian?


...


[1]: モデルCのみが得意とする問題
  What studio album did Kanye West record with Roc-A-Fella Records and soul singer Dwele?


,ansA,ansB,ansC,question,word_count,char_count,sentence_count,avg_word_length,avg_sentence_lenght
5a83aaeb5542996488c2e483,0,0,1,What studio album did Kanye West record with R...,14,74,1,5.285714,14
5a7c38c955429935c91b5132,0,0,1,What characteristic meaning it's transferable ...,14,93,1,6.642857,14



[5]: モデルA･Cが得意とする問題
  Did Greg Costikyan have the same profession as John Dolmayan?  


,ansA,ansB,ansC,question,word_count,char_count,sentence_count,avg_word_length,avg_sentence_lenght
5a8e43455542990e94052aa1,1,0,1,Did Greg Costikyan have the same profession as...,12,52,1,4.333333,12
5adc8c545542994734353734,1,0,1,Are Wilco Melissant and Todd Field both involv...,16,89,3,5.562500,16



temp
Index(['word_count', 'char_count', 'avg_word_length', 'avg_sentence_lenght'], dtype='object')


In [ ]:
for i, df_ in enumerate(temp):
  print('[{}]: {}'.format(i, comment[i]))
  print(df_)

[0]: 全モデルが苦手とする問題
                          word_count  char_count  avg_word_length  \
5ae16ea85542990adbacf790          13          73         5.615385   
5ac10832554299012d1db687          33         162         4.909091   
5ae2c4b8554299495565db31          13          50         3.846154   
5a78d38455429970f5fffd94          21         103         4.904762   
5adce4f65542992c1e3a2474          14          89         6.357143   
...                              ...         ...              ...   
5a72644b5542997f8278399a          17         105         6.176471   
5a7af08655429931da12c989           8          50         6.250000   
5ae1d1a55542997f29b3c138          34         136         4.000000   
5ae5669755429960a22e02ec          28         126         4.500000   
5ac3d667554299076e296ca5          11          71         6.454545   

                          avg_sentence_lenght  
5ae16ea85542990adbacf790                   13  
5ac10832554299012d1db687                   33  
5ae2c4b85

In [ ]:
idx = [0, 7, 4, 6, 2, 3, 1, 5]
table = []
means = []
vars  = []
for i in range(len(temp)):
  line = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  mean = 0
  var = 0
  for j, row in temp[i].iterrows():
    mean += row.word_count
    line[(int)(row.word_count / 5)] += 1
  mean /= len(temp[i])
  for j, row in temp[i].iterrows():
    var += (row.word_count - mean)**2
  var /= len(temp[i])
  means.append(mean)
  vars.append(var)
  table.append(line)

for i in idx:
  print('[{}]'.format(i))
  print('  全単語数: {}'.format(round(means[i] * len(temp[i]))))
  print('  文章の数: {}'.format(len(temp[i])))

print('\n   ', end='')
for i in idx:
  print('   [{}]'.format(i), end='')
print('\n----------------------------------------------------')
print('mean  ', end='')

for i in idx:
  print('{:.1f}  '.format(means[i]), end='')
print('\nvar   ', end='')
for i in idx:
  print('{:.1f}  '.format(vars[i]), end='')

[0]
  全単語数: 50542
  文章の数: 3200
[7]
  全単語数: 32694
  文章の数: 2043
[4]
  全単語数: 4500
  文章の数: 288
[6]
  全単語数: 6193
  文章の数: 386
[2]
  全単語数: 9715
  文章の数: 622
[3]
  全単語数: 4653
  文章の数: 299
[1]
  全単語数: 4257
  文章の数: 268
[5]
  全単語数: 4147
  文章の数: 270

      [0]   [7]   [4]   [6]   [2]   [3]   [1]   [5]
----------------------------------------------------
mean  15.8  16.0  15.6  16.0  15.6  15.6  15.9  15.4  
var   29.4  32.1  32.4  32.2  32.0  26.9  34.3  27.5  

In [ ]:
print('       0    5   10   15   20   25   30   35   40   45')
print('------------------------------------------------------')
sums = []
idx = [0, 7, 4, 6, 2, 3, 1, 5]
for i in idx:
  sum = 0
  print('[{}]'.format(i), end='')
  for data in table[i]:
    print('{:5d}'.format(data), end='')
    sum += data
  sums.append(sum)
  print('   (mean:{:5.1f},   var:{:5.1f})'.format(means[i], vars[i]))

print()
print('         0      5     10     15     20     25     30     35     40     45')
print('--------------------------------------------------------------------------')
for i in idx:
  print('[{}]'.format(i), end='')
  for data in table[i]:
    print('  {:4.1f}%'.format(100*data/sums[i]), end='')
  print('   (mean:{:5.1f}%,   var:{:5.1f}%)'.format(100*means[i]/sums[i], 100*vars[i]/sums[i]))


       0    5   10   15   20   25   30   35   40   45
------------------------------------------------------
[0]    0  307 1164 1045  450  170   49   12    2    1   (mean: 15.8,   var: 29.4)
[7]    0  206  708  639  333  103   40   11    2    1   (mean: 16.0,   var: 32.1)
[4]    0   28  116   85   36   14    7    1    1    0   (mean: 15.6,   var: 32.4)
[6]    0   34  150  114   54   20   13    1    0    0   (mean: 16.0,   var: 32.2)
[2]    0   61  249  188   70   39    9    5    1    0   (mean: 15.6,   var: 32.0)
[3]    0   27  115   97   39   14    7    0    0    0   (mean: 15.6,   var: 26.9)
[1]    0   29   99   79   31   22    7    1    0    0   (mean: 15.9,   var: 34.3)
[5]    0   31  104   80   40   10    5    0    0    0   (mean: 15.4,   var: 27.5)

         0      5     10     15     20     25     30     35     40     45
--------------------------------------------------------------------------
[0]   0.0%   9.6%  36.4%  32.7%  14.1%   5.3%   1.5%   0.4%   0.1%   0.0%   (mean:  0

In [ ]:
char_table = []
char_means = []
char_vars  = []
for i in range(len(temp)):
  line = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  char_mean = 0
  char_var = 0
  for j, row in temp[i].iterrows():
    char_mean += row.word_count
    line[(int)(row.char_count / 20)] += 1
  char_mean /= round(means[i] * len(temp[i]))
  for j, row in temp[i].iterrows():
    char_var += (row.char_count - char_mean)**2
  char_var /= round(len(temp[i]) * len(temp[i]))
  char_means.append(char_mean)
  char_vars.append(char_var)
  char_table.append(line)

for i in idx:
  print('[{}]'.format(i))
  print('  全文字数: {}'.format(round(means[i] * len(temp[i])) * char_means[i]))
  print('  全単語数: {}'.format(round(means[i] * len(temp[i]))))

print('\n    ', end='')
for i in idx:
  print('  [{}]'.format(i), end='')
print('\n----------------------------------------------------')
print('mean', end='')

for i in idx:
  print('{:5.1f}'.format(char_means[i]), end='')
print('\nvar ', end='')
for i in idx:
  print('{:5.1f}'.format(char_vars[i]), end='')

[0]
  全文字数: 50542.0
  全単語数: 50542
[7]
  全文字数: 32694.0
  全単語数: 32694
[4]
  全文字数: 4500.0
  全単語数: 4500
[6]
  全文字数: 6193.0
  全単語数: 6193
[2]
  全文字数: 9715.0
  全単語数: 9715
[3]
  全文字数: 4653.0
  全単語数: 4653
[1]
  全文字数: 4257.0
  全単語数: 4257
[5]
  全文字数: 4147.0
  全単語数: 4147

      [0]  [7]  [4]  [6]  [2]  [3]  [1]  [5]
----------------------------------------------------
mean  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
var   2.0  3.3 23.1 17.6 10.5 21.7 25.5 23.0

In [ ]:
print('       0   20   40   60   80  100  120  140  160  180  200  220  240')
print('---------------------------------------------------------------------')
char_sums = []
idx = [0, 7, 4, 6, 2, 3, 1, 5]
for i in range(len(temp)):
  char_sum = 0
  for data in char_table[i]:
    char_sum += data
  char_sums.append(char_sum)

for i in idx:
  print('[{}]'.format(i), end='')
  for data in char_table[i]:
    print('{:5d}'.format(data), end='')
  print('   (mean:{:5.1f},   var:{:5.1f})'.format(char_means[i], char_vars[i]))

print()
print('     0      20      40      60      80     100     120     140     160     180     200     220     240     260')
print('----------------------------------------------------------------------------------------------------------------')
for i in idx:
  print('[{}]'.format(i), end='')
  for data in char_table[i]:
    print('  {:5.1f}%'.format(100*data/char_sums[i]), end='')
  print('   (mean:{:5.1f}%,   var:{:5.1f}%)'.format(100*means[i]/char_sums[i], 100*vars[i]/char_sums[i]))


       0   20   40   60   80  100  120  140  160  180  200  220  240
---------------------------------------------------------------------
[0]    0   93  802 1020  713  343  144   53   20   12    0    0    0   (mean:  1.0,   var:  2.0)
[7]    0   63  500  652  434  224  105   39   19    3    1    2    1   (mean:  1.0,   var:  3.3)
[4]    0    7   73   93   61   27   17    4    5    0    1    0    0   (mean:  1.0,   var: 23.1)
[6]    0   15   88  139   67   41   15   17    2    2    0    0    0   (mean:  1.0,   var: 17.6)
[2]    0   14  162  222  118   53   29   15    4    2    3    0    0   (mean:  1.0,   var: 10.5)
[3]    0    5   82   96   65   25   17    7    2    0    0    0    0   (mean:  1.0,   var: 21.7)
[1]    0    5   76   79   55   24   17   10    1    1    0    0    0   (mean:  1.0,   var: 25.5)
[5]    0    9   77   79   61   24   13    5    2    0    0    0    0   (mean:  1.0,   var: 23.0)

     0      20      40      60      80     100     120     140     160     180     2

### ショートカットここまで

In [ ]:
file_name  = 'GOOD_Ans_to{}.txt'.format(pred2_path[-11:-5])
file_title = '{}_to_{}_得意になった質問'.format(pred1_path[-11:-5], pred2_path[-11:-5])
writer(better_answer.head(5), output='file', file_name=file_name, file_title=file_title)

file_name  = 'BAD_Ans_to{}.txt'.format(pred2_path[-11:-5])
file_title = '{}_to_{}_苦手になった質問'.format(pred1_path[-11:-5], pred2_path[-11:-5])
writer(worse_answer.head(5), output='file', file_name=file_name, file_title=file_title)

file_name  = 'NORMAL_GOOD_Ans_to{}.txt'.format(pred2_path[-11:-5])
file_title = '{}_to_{}_もともと得意な質問'.format(pred1_path[-11:-5], pred2_path[-11:-5])
writer(normal_answer_good.head(5), output='file', file_name=file_name, file_title=file_title)

file_name  = 'NORMAL_BAD_Ans_to{}.txt'.format(pred2_path[-11:-5])
file_title = '{}_to_{}_もともと苦手な質問'.format(pred1_path[-11:-5], pred2_path[-11:-5])
writer(normal_answer.head(5), output='file', file_name=file_name, file_title=file_title, print_true_ans=True)

output mode      : file
file_name        : GOOD_Ans_toLLLLLL.txt
file_title       : RRRRRR_to_LLLLLL_得意になった質問
translate        : True
print_true_answer: False


  5%|▍         | 21/448 [00:21<07:16,  1.02s/it]


KeyboardInterrupt: ignored

In [ ]:
print('2/3 = {}'.format(2/3))

2/3 = 0.6666666666666666


In [ ]:
print('2/3 = {:.2}'.format(2/3))

2/3 = 0.67


In [ ]:
file_name  = '[EN]GOOD_Ans_to{}.txt'.format(pred2_path[-11:-5])
file_title = '{}_to_{}_得意になった質問'.format(pred1_path[-11:-5], pred2_path[-11:-5])
writer(better_answer.head(5), output='file', file_name=file_name, translate=False, file_title=file_title)

file_name  = '[EN]BAD_Ans_to{}.txt'.format(pred2_path[-11:-5])
file_title = '{}_to_{}_苦手になった質問'.format(pred1_path[-11:-5], pred2_path[-11:-5])
writer(worse_answer.head(5), output='file', file_name=file_name, translate=False, file_title=file_title)

file_name  = '[EN]NORMAL_GOOD_Ans_to{}.txt'.format(pred2_path[-11:-5])
file_title = '{}_to_{}_もともと得意な質問'.format(pred1_path[-11:-5], pred2_path[-11:-5])
writer(normal_answer_good.head(5), output='file', file_name=file_name, translate=False, file_title=file_title)

file_name  = '[EN]NORMAL_BAD_Ans_to{}.txt'.format(pred2_path[-11:-5])
file_title = '{}_to_{}_もともと苦手な質問'.format(pred1_path[-11:-5], pred2_path[-11:-5])
writer(normal_answer.head(5), output='file', file_name=file_name, file_title=file_title, translate=False, print_true_ans=True)

output mode      : file
file_name        : [EN]GOOD_Ans_toRRLLLL.txt
file_title       : RRRRRR_to_RRLLLL_得意になった質問
translate        : False
print_true_answer: False
output mode      : file
file_name        : [EN]BAD_Ans_toRRLLLL.txt
file_title       : RRRRRR_to_RRLLLL_苦手になった質問
translate        : False
print_true_answer: False
output mode      : file
file_name        : [EN]NORMAL_GOOD_Ans_toRRLLLL.txt
file_title       : RRRRRR_to_RRLLLL_もともと得意な質問
translate        : False
print_true_answer: False
output mode      : file
file_name        : [EN]NORMAL_BAD_Ans_toRRLLLL.txt
file_title       : RRRRRR_to_RRLLLL_もともと苦手な質問
translate        : False
print_true_answer: True


In [ ]:
file_name  = 'GOOD_SP_to{}.txt'.format(pred2_path[-11:-5])
file_title = '{}_to_{}_得意になった質問(SP)'.format(pred1_path[-11:-5], pred2_path[-11:-5])
#writer(better_sp.head(5), output='file', file_name=file_name, file_title=file_title)

file_name  = 'BAD_SP_to{}.txt'.format(pred2_path[-11:-5])
file_title = '{}_to_{}_苦手になった質問(SP)'.format(pred1_path[-11:-5], pred2_path[-11:-5])
#writer(worse_sp.head(5), output='file', file_name=file_name, file_title=file_title)

file_name  = 'NORMAL_GOOD_SP_to{}.txt'.format(pred2_path[-11:-5])
file_title = '{}_to_{}_もともと得意な質問(SP)'.format(pred1_path[-11:-5], pred2_path[-11:-5])
writer(normal_sp_good.head(5), output='file', file_name=file_name, file_title=file_title)

file_name  = 'NORMAL_BAD_SP_to{}.txt'.format(pred2_path[-11:-5])
file_title = '{}_to_{}_もともと苦手な質問(SP)'.format(pred1_path[-11:-5], pred2_path[-11:-5])
writer(normal_sp.head(5), output='file', file_name=file_name, file_title=file_title, print_true_ans=True)

output mode      : file
file_name        : NORMAL_GOOD_SP_toRRLLLL.txt
file_title       : RRRRRR_to_RRLLLL_もともと得意な質問(SP)
translate        : True
print_true_answer: False


100%|██████████| 362/362 [00:48<00:00,  7.44it/s]


output mode      : file
file_name        : NORMAL_BAD_SP_toRRLLLL.txt
file_title       : RRRRRR_to_RRLLLL_もともと苦手な質問(SP)
translate        : True
print_true_answer: True


100%|██████████| 498/498 [07:28<00:00,  1.11it/s]


In [ ]:
file_name  = '[EN]GOOD_SP_to{}.txt'.format(pred2_path[-11:-5])
file_title = '{}_to_{}_得意になった質問(SP)'.format(pred1_path[-11:-5], pred2_path[-11:-5])
writer(better_sp.head(5), output='file', file_name=file_name, file_title=file_title, translate=False)

file_name  = '[EN]BAD_SP_to{}.txt'.format(pred2_path[-11:-5])
file_title = '{}_to_{}_苦手になった質問(SP)'.format(pred1_path[-11:-5], pred2_path[-11:-5])
writer(worse_sp.head(5), output='file', file_name=file_name, file_title=file_title, translate=False)

file_name  = '[EN]NORMAL_GOOD_SP_to{}.txt'.format(pred2_path[-11:-5])
file_title = '{}_to_{}_もともと得意な質問(SP)'.format(pred1_path[-11:-5], pred2_path[-11:-5])
writer(normal_sp_good.head(5), output='file', file_name=file_name, file_title=file_title, translate=False)

file_name  = '[EN]NORMAL_BAD_SP_to{}.txt'.format(pred2_path[-11:-5])
file_title = '{}_to_{}_もともと苦手な質問(SP)'.format(pred1_path[-11:-5], pred2_path[-11:-5])
writer(normal_sp.head(5), output='file', file_name=file_name, file_title=file_title, print_true_ans=True, translate=False)

output mode      : file
file_name        : [EN]GOOD_SP_toLLLLLL.txt
file_title       : RRRRRR_to_LLLLLL_得意になった質問(SP)
translate        : False
print_true_answer: False
output mode      : file
file_name        : [EN]BAD_SP_toLLLLLL.txt
file_title       : RRRRRR_to_LLLLLL_苦手になった質問(SP)
translate        : False
print_true_answer: False
output mode      : file
file_name        : [EN]NORMAL_GOOD_SP_toLLLLLL.txt
file_title       : RRRRRR_to_LLLLLL_もともと得意な質問(SP)
translate        : False
print_true_answer: False
output mode      : file
file_name        : [EN]NORMAL_BAD_SP_toLLLLLL.txt
file_title       : RRRRRR_to_LLLLLL_もともと苦手な質問(SP)
translate        : False
print_true_answer: True


## 段落検索

### 関数たち

In [ ]:
import re

# stop_num: 0で無効
def search_rows(search_word, stop_num=0, printer=False):
  searched_rows = []
  i = 0
  for index, row in gold.iterrows():
    search_flag = 0
    for context in row['context']:
      j = 0
      for text in context:
        if j == 0:
          if len(re.findall(search_word, text)) > 0:
            search_flag = 1
          if printer == True:
            print('title: [{}]'.format(text))
        else:
          for k in range(len(text)):
            if len(re.findall(search_word, text[k])) > 0:
              search_flag = 1
            if printer == True:
              print('[{}] {}'.format(k, text[k]))
        j += 1
  
    if len(re.findall(search_word, row['question'])) > 0:
      search_flag = 1
  
    if search_flag == 1:
      searched_rows.append((index, row))
    i += 1
    if i == stop_num:
      break
  if stop_num != 0:
    print('(!) stop_numが有効です.')
  return searched_rows
 

In [ ]:
def rows_show(rows, msg=''):
  for row in rows:
    print('\n[id: {}]'.format(row[0]))
    for line in row[1]['context']:
      print('"{}"'.format(line[0]))
      i = 0
      for text in line[1]:
        if i == 0:
          print('   {}'.format(text))
        else:
          print('  {}'.format(text))
        i += 1
  print('\n{}'.format(msg))

### 入力

In [ ]:
# stop_num...サンプルを先頭からnum個に限定します（0で無効）
stop_num = 0
search_word = input('seach word or phrase: ')
rows = search_rows(search_word, stop_num)
rows_show(rows, '** {} context in search results **'.format(len(rows)))

seach word or phrase: Jean-Paul Sartre or George Bernard

[id: 4157]
"Saint Genet"
   Saint Genet, Actor and Martyr (French: "Saint Genet, comédien et martyr" ) is a book by the French philosopher Jean-Paul Sartre about the writer Jean Genet especially on his "The Thief's Journal".
   It was first published in 1952.
   Sartre described it as an attempt "to prove that genius is not a gift but the way out that one invents in desperate cases."
   Sartre also based his character Goetz in his play "The Devil and the Good Lord" (1951) on his analysis of Genet's psychology and morality.
   Sartre has been credited by David M. Halperin with providing, "a brilliant, subtle, and thoroughgoing study of the unique subjectivity and gender positioning of gay men".
"The Transcendence of the Ego"
   The Transcendence of the Ego (French: "La Transcendance de l'ego: Esquisse d'une description phénomenologique" ) is a philosophical and psychological essay written by Jean-Paul Sartre in 1934 and published